# Sample workflow: 1990 block group parts to 2010 tracts

## Starting from a subset of 2010 Delaware blocks

For further background information see:
* **Schroeder, J. P**. 2007. *Target-density weighting interpolation and uncertainty evaluation for temporal analysis of census data*. Geographical Analysis 39 (3):311–335.

#### NHGIS [block crosswalks](https://www.nhgis.org/user-resources/geographic-crosswalks)

In [1]:
%load_ext watermark
%watermark

2020-06-13T14:43:33-04:00

CPython 3.7.6
IPython 7.15.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [2]:
import nhgisxwalk
import inspect
import numpy
import pandas

%load_ext autoreload
%autoreload 2
%watermark -w
%watermark -iv

watermark 2.0.2
numpy      1.18.5
nhgisxwalk 0.0.2
pandas     1.0.4



### Source and target years for the crosswalk

In [3]:
source_year, target_year = "1990", "2010"

### Source-target building base

In [4]:
subset_data_dir = "../testing_data_subsets"
base_xwalk_name = "/nhgis_blk%s_blk%s_gj.csv.zip" % (source_year, target_year)
base_xwalk_file = subset_data_dir + base_xwalk_name
data_types = nhgisxwalk.str_types(["GJOIN%s"%source_year, "GJOIN%s"%target_year])
base_xwalk = pandas.read_csv(base_xwalk_file, index_col=0, dtype=data_types)
base_xwalk.head()

,GJOIN1990,GJOIN2010,WEIGHT,PAREA_VIA_BLK00
0,NaN,G10000100432021078,0.0,0.0
1,NaN,G10000100432023014,0.0,0.0
2,NaN,G10000100432023015,0.0,0.0
3,NaN,G10000109900000011,0.0,0.0
4,NaN,G10000109900000012,0.0,0.0


### Source supplementary summary data (special case for 1990)

In [5]:
supp_source_name = "%s_blck_grp_598_103.csv.zip" % source_year
supp_source_file = subset_data_dir + supp_source_name

### Convenience code shorthand/lookup

In [6]:
print(inspect.getsource(nhgisxwalk.valid_geo_shorthand))

def valid_geo_shorthand(shorthand_name=True):
    """Shorthand lookups for census geographies."""
    lookup = {
        "blk": "block",
        "bgp": "block group part",
        "bkg": "block group",
        "trt": "tract",
        "cty": "county",
    }
    if not shorthand_name:
        lookup = {v: k for k, v in lookup.items()}
    return lookup



In [7]:
nhgisxwalk.valid_geo_shorthand(shorthand_name=False)

{'block': 'blk',
 'block group part': 'bgp',
 'block group': 'bkg',
 'tract': 'trt',
 'county': 'cty'}

### Instantiate an `nhgisxwalk.GeoCrossWalk` object
##### see [nhgisxwalk.GeoCrossWalk](https://github.com/jGaboardi/nhgisxwalk/blob/92b4fe55de0a9c53d0315dcda8ec121faaf20aef/nhgisxwalk/geocrosswalk.py#L19) for full details

In [8]:
nhgisxwalk.desc_code_1990

{'Persons': {'Persons': 'Universe',
  'NP1': 'Source code',
  'ET1': 'NHGIS code',
  'Total': 'ET1001'},
 'Families': {'Families': 'Universe',
  'NP2': 'Source code',
  'EUD': 'NHGIS code',
  'Total': 'EUD001'},
 'Households': {'Households': 'Universe',
  'NP3': 'Source code',
  'EUO': 'NHGIS code',
  'Total': 'EUO001'},
 'Housing Units': {'Housing Units': 'Universe',
  'NH1': 'Source code',
  'ESA': 'NHGIS code',
  'Total': 'ESA001'}}

In [9]:
input_vars = [
    nhgisxwalk.desc_code_1990["Persons"]["Total"],
    nhgisxwalk.desc_code_1990["Families"]["Total"],
    nhgisxwalk.desc_code_1990["Households"]["Total"],
    nhgisxwalk.desc_code_1990["Housing Units"]["Total"]
]
input_vars

['ET1001', 'EUD001', 'EUO001', 'ESA001']

In [10]:
input_var_tags = ["pop", "fam", "hh", "hu"]

In [11]:
subset_state = "10"
bgp1990_to_trt2010 = nhgisxwalk.GeoCrossWalk(
    base_xwalk,
    source_year=source_year,
    target_year=target_year,
    source_geo="bgp",
    target_geo="trt",
    base_source_table=subset_data_dir+"/1990_block.csv.zip",
    supp_source_table=subset_data_dir+"/1990_blck_grp_598_103.csv.zip",
    input_var=input_vars,
    weight_var=input_var_tags,
    stfips=subset_state,
    keep_base=True,
    add_geoid=True
)
bgp1990_to_trt2010.xwalk[688:697]

,bgp1990gj,trt2010gj,trt2010ge,wt_pop,wt_fam,wt_hh,wt_hu
688,G100003093996775800002009999999916012,G1000030000200,10003000200,1.0,1.0,1.0,1.0
689,G100003093996775800002009999999916013,G1000030000200,10003000200,1.0,1.0,1.0,1.0
690,G100003093996775800002009999999916013,G1000030010800,10003010800,0.0,0.0,0.0,0.0
691,G100003093996775800002009999999916014,G1000030000200,10003000200,1.0,1.0,1.0,1.0
692,G100003093996775800002009999999916015,G1000030000200,10003000200,1.0,1.0,1.0,1.0
693,G100003093996775800002009999999916016,G1000030000200,10003000200,1.0,1.0,1.0,1.0
694,G100003093996775800002009999999916016,G1000030000300,10003000300,0.0,0.0,0.0,0.0
695,G100003093996775800003009999999916011,G1000030000300,10003000300,1.0,1.0,1.0,1.0
696,G100003093996775800003009999999916012,G1000030000300,10003000300,1.0,1.0,1.0,1.0


### Write crosswalk to a `.csv`

In [12]:
res_dir = "../../results/"
bgp1990_to_trt2010.xwalk_to_csv(path=res_dir)

### Read crosswalk from a `.csv`

In [13]:
fname = res_dir + "bgp1990gj_to_trt2010gj_%s" % subset_state
bgp1990_to_trt2010 = nhgisxwalk.GeoCrossWalk.xwalk_from_csv(fname)
bgp1990_to_trt2010[688:697]

,bgp1990gj,trt2010gj,trt2010ge,wt_pop,wt_fam,wt_hh,wt_hu
688,G100003093996775800002009999999916012,G1000030000200,10003000200,1.0,1.0,1.0,1.0
689,G100003093996775800002009999999916013,G1000030000200,10003000200,1.0,1.0,1.0,1.0
690,G100003093996775800002009999999916013,G1000030010800,10003010800,0.0,0.0,0.0,0.0
691,G100003093996775800002009999999916014,G1000030000200,10003000200,1.0,1.0,1.0,1.0
692,G100003093996775800002009999999916015,G1000030000200,10003000200,1.0,1.0,1.0,1.0
693,G100003093996775800002009999999916016,G1000030000200,10003000200,1.0,1.0,1.0,1.0
694,G100003093996775800002009999999916016,G1000030000300,10003000300,0.0,0.0,0.0,0.0
695,G100003093996775800003009999999916011,G1000030000300,10003000300,1.0,1.0,1.0,1.0
696,G100003093996775800003009999999916012,G1000030000300,10003000300,1.0,1.0,1.0,1.0


-----------------------------------------------